Esta notebook contiene bloques de código útiles para realizar Q-learning en el entorno "Pendulum"

In [1]:
import numpy as np
from pendulum_env_extended import PendulumEnvExtended
import random 

In [2]:
alpha = 0.1
gamma = 0.99
epsilon = 0.1
n_episodes = 100
epsilon_variability = 0.8
cant_Buckets = 10

In [3]:
env = PendulumEnvExtended(render_mode='rgb_array')

Discretización de los estados

In [4]:
x_space = np.linspace(-1, 1, cant_Buckets)
y_space = np.linspace(-1, 1, cant_Buckets)
vel_space = np.linspace(-8, 8, 100)
x_space

array([-1.        , -0.77777778, -0.55555556, -0.33333333, -0.11111111,
        0.11111111,  0.33333333,  0.55555556,  0.77777778,  1.        ])

Obtener el estado a partir de la observación

In [5]:
def get_state(obs):
    x, y, vel = obs
    x_bin = np.digitize(x, x_space)
    y_bin = np.digitize(y, y_space)
    vel_bin = np.digitize(vel, vel_space)
    return x_bin, y_bin, vel_bin

In [6]:
state = get_state(np.array([-0.4, 0.2, 0.3])) #mapeo de estado. Nos dice el bin en el que estamos
state

(3, 6, 52)

Discretización de las acciones

In [7]:
actions = list(np.linspace(-2, 2, cant_Buckets)) #minimo, maximo y cuantos bins
actionBuckets=np.linspace(-2, 2, cant_Buckets)
actions

[-2.0,
 -1.5555555555555556,
 -1.1111111111111112,
 -0.6666666666666667,
 -0.22222222222222232,
 0.22222222222222232,
 0.6666666666666665,
 1.1111111111111107,
 1.5555555555555554,
 2.0]

In [8]:
def getActions(action):
    return np.digitize(action,actionBuckets)

In [9]:
def get_sample_action():
    return random.choice(actions)

Inicilización de la tabla Q

In [10]:
Q = np.zeros((len(x_space) + 1, len(y_space) + 1, len(vel_space) + 1, len(actions)))
Q

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

Obtención de la acción a partir de la tabla Q

In [11]:
def optimal_policy(state, Q):
    action = actions[np.argmax(Q[state])]
    return action

Epsilon-Greedy Policy

In [12]:
def epsilon_greedy_policy(state, Q, epsilon=0.1):
    explore = np.random.binomial(1, epsilon)
    # explore
    if explore:
        action = get_sample_action()
        print('explore')
    # exploit
    else:
        action = optimal_policy(state, Q)
        print('exploit')
        
    return action

Ejemplo de episodio 

In [13]:
%pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [14]:
obs,_ = env.reset()
print(obs)
done = False
total_reward = 0
while not done:
    state = get_state(obs)
    print('state', state)
    action = epsilon_greedy_policy(state, Q, 0.5)
    action_idx = actions.index(action)
    # Acción del ambiente
    real_action = np.array([action])

    obs, reward, done, _, _ = env.step(real_action)
    next_state = get_state(obs)
    
    total_reward += reward
    print('->', state, action, reward, obs, done)
    env.render()
print('total_reward', total_reward)

[-0.3192592   0.9476674   0.38504696]
state (4, 9, 52)
exploit
-> (4, 9, 52) -2.0 -3.612671426239341 [-0.35670412  0.9342174   0.7957975 ] False
state (3, 9, 55)
explore
-> (3, 9, 55) -2.0 -3.8136207599543805 [-0.4119204   0.91121984  1.1964606 ] False
state (3, 9, 57)
explore
-> (3, 9, 57) 0.22222222222222232 -4.124650365613566 [-0.49707192  0.86770934  1.9132088 ] False
state (3, 9, 62)
exploit
-> (3, 9, 62) -2.0 -4.742390340889008 [-0.59190524  0.80600756  2.2639909 ] False
state (2, 9, 64)
exploit
-> (2, 9, 64) -2.0 -5.37513752862884 [-0.69025785  0.72356343  2.5684965 ] False
state (2, 8, 66)
explore
-> (2, 8, 66) -2.0 -6.104934785812543 [-0.7848189   0.61972517  2.8111691 ] False
state (1, 8, 67)
explore
-> (1, 8, 67) 2.0 -6.9109862294992475 [-0.8825237   0.47026792  3.575963  ] False
state (1, 7, 72)
explore
-> (1, 7, 72) 2.0 -8.31584618445714 [-0.961562    0.27458793  4.228664  ] False
state (1, 6, 76)
explore
-> (1, 6, 76) -0.22222222222222232 -9.9874492516866 [-0.99831283  0.

In [15]:
%pip install sklearn


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [16]:
import wandb
from wandb.sklearn import plot_precision_recall, plot_feature_importances
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc

def train_policy():
        try:
            with wandb.init() as run:
                config = run.config
                alpha = config.alpha
                gamma = config.gamma
                epsilon = config.epsilon
                epsilon_variability = config.epsilonVariability
                episodes = config.episodes
                total_rewards = []
                total_reward_promedio = []

                for episode in range(episodes):
                    obs, _ = env.reset()
                    done = False
                    total_reward = 0
                    step_count = 0

                    while not done:
                        state = obs
                        epsilon = max(epsilon - epsilon_variability * epsilon, 0)  # Decay epsilon
                        action = epsilon_greedy_policy(get_state(state), Q, epsilon)
                        obs, reward, done, _, _ = env.step([action])
                        total_reward_promedio.append(reward)
                        newState = get_state(state)
                        binAction = getActions(action)
                        Q[newState[0],newState[1],newState[2],binAction] += alpha * (reward + gamma * np.max(Q[get_state(obs)]) - Q[newState[0],newState[1],newState[2],binAction])
                        total_reward += reward
                        step_count += 1
                        wandb.log({'episode_reward': reward})

                    total_rewards.append(total_reward)
                    last_ten_rewards = total_reward_promedio[-10:]
                    last_rewards_mean = np.mean(last_ten_rewards)
                    wandb.log({'train_avg_reward': last_rewards_mean})
        except Exception as e:
                print(f"An error occurred: {e}")
        finally:
                wandb.finish()

In [17]:
import wandb

# Configuración del sweep
sweep_configuration = {
    "method": "random",
    "metric": {"goal": "maximize", "name": 'train_avg_reward'},
    "parameters": {
        "episodes": {"max": 10000, "min": 9999},
        "alpha": {"max": 0.8, "min": 0.5},
        "gamma": {"max": 0.6, "min": 0.2},
        "epsilon": {"max": 0.8, "min": 0.1},
        "epsilonVariability": {"max": 0.5, "min": 0.35}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="pendulum-sweep")
wandb.agent(sweep_id, function=train_policy)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


ServiceStartPortError: Failed to allocate port for wandb service: [Errno 13] Permission denied: 'C:\\Users\\maria\\AppData\\Local\\Temp\\tmp9wygvjdu\\port-16548.txt'.

In [ ]:
import wandb
from wandb.sklearn import plot_precision_recall, plot_feature_importances
from wandb.sklearn import plot_class_proportions, plot_learning_curve, plot_roc

def train_finalPolicy():
    try:
            #Cambiar valores de alpha, gamma, epsilon, epsilon_variability y episodes
            alpha = 0.8894046109188853
            gamma = 0.5333435999129466
            epsilon = 0.7391381811833376
            epsilon_variability = 0.4889625280062114
            episodes = 10000

            env = PendulumEnvExtended()
            Q = np.zeros([env.observation_space.n, env.action_space.n])

            total_rewards = []

            for episode in range(episodes):
                obs, _ = env.reset()
                done = False
                total_reward = 0
                step_count = 0

                while not done:
                    state = obs
                    epsilon = max(epsilon - epsilon_variability * epsilon, 0)  # Decay epsilon
                    action = epsilon_greedy_policy(state, Q, epsilon)
                    obs, reward, done, _, _ = env.step(action)
                    Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[obs]) - Q[state, action])
                    total_reward += reward
                    step_count += 1

                total_rewards.append(total_reward)
                print(f"Episode {episode} - Total reward: {total_reward}, avg_total_reward: {np.mean(total_rewards)}")
            return Q


    except Exception as e:
        print(f"An error occurred: {e}")